In [ ]:
%cd indicTrans/

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
import pandas as pd
from datasets import load_dataset, DatasetDict, load_dataset
from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from indictrans import Transliterator
from indicTrans.inference.engine import Model
from nltk.corpus import words
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split
import demoji
import re
from tqdm.notebook import tqdm
from IPython.display import clear_output
import enchant
import warnings
import logging

In [ ]:
# Change this for debugging
warnings.filterwarnings('once')
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
%cd ..
# train = pd.read_csv("./tamil_offensive_train.tsv", sep="\t")
# test = pd.read_csv("./tamil_offensive_test.tsv", sep="\t")
# train = pd.read_csv("./new/train.csv")
# validation = pd.read_csv("./new/validation.csv")
%cd indicTrans/

In [ ]:
indic2en_model = Model(expdir='../en-indic')
transliterator = Transliterator(source='eng', target='tam', build_lookup=True)
d = enchant.Dict("en_US")

In [ ]:
def cleanup_text(text):
    # remove punctuation
    text = re.sub('[!#?,.:";]', ' ', text)
    # remove multiple spaces
    text = re.sub(r' +', ' ', text)
    # remove newline
    text = re.sub(r'\n', ' ', text)
    return text

In [ ]:
count = 0
def preprocess_line(line):
    translated = ""
    global d
    global count
    count += 1

    for text in line.split():
        stripped_text = text.strip("[$&+,:;=?@#|'<>.^*()%!-]\\/ ")
        
        if stripped_text == "": continue
        
        if d.check(stripped_text):
            translated += indic2en_model.translate_paragraph(stripped_text, "en", "ta")
        else:
            translated += stripped_text
        translated += " "
    translated = transliterator.transform(translated)
    
    return translated

In [ ]:
def postprocess():
    data = pd.read_csv("./tamil_offensive_train.tsv", sep="\t")
    labels = data["label"]
    texts = data["text"]
    texts = [" ".join([text for text in sentence.split() if text[0] != "@" and text[0] != "#"]) for sentence in texts]
    texts = [demoji.replace(text) for text in texts]
    texts = [re.sub(r'http\S+', '', text) for text in texts]
    empty_indices = []
    for i in range(len(texts)):
        if texts[i].strip() == "":
            empty_indices.append(i)
    
    texts = [text for i, text in enumerate(texts) if i not in empty_indices]
    labels = [label for i, label in enumerate(labels) if i not in empty_indices]
    texts = [cleanup_text(text) for text in texts]
    df = pd.DataFrame({
        "text": texts,
        "label": labels
    })
    train, test = train_test_split(df, test_size=0.2)
    train.to_csv("./dataset/train.csv", index=False)
    test.to_csv("./dataset/validation.csv", index=False)

In [ ]:
warnings.filterwarnings('ignore')
preprocessed_sentences_validation = []
for index, row in tqdm(validation.iterrows()):
    preprocessed_sentences_validation.append(preprocess_line(row["text"]))

In [ ]:
validation["text"] = preprocessed_sentences_validation

In [ ]:
validation.to_csv("../new/validation_processed.csv", index=False)

In [ ]:
train["text"] = preprocessed_sentences

In [ ]:
train.to_csv("../new/train_processed.csv", index=False)

In [ ]:
postprocess()